In [1]:
import sys
sys.path.append('./')
from utils import get_sequences

import os
os.environ["BRT_ENV"] = 'prod'
import json
import random
import brtdevkit
print(brtdevkit.__version__)
brtdevkit.log = 'info'
import os
os.environ['AWS_PROFILE'] = 'default'
USERNAME = 'li.yu'

import ast
import time
import numpy as np
import pandas as pd
from datetime import datetime, timezone

# from brtdevkit.core.db.athena import AthenaClient, Table
# from brtdevkit.data import Image, Dataset
# from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
# from aletheia_dataset_creator.config.dataset_config import *

# from jupiterdata.utils.dataset import query_db
from jupiterdata.config.dataset_config import *
from jupiterdata.utils.dataset import *

pd.set_option('display.max_columns', None)

# %matplotlib inline
%load_ext autoreload
%autoreload 2

8.11.2


In [2]:
""" # run this on cmd: 
eval "$(/home/li.yu/anaconda3/bin/conda shell.bash hook)"
conda activate query
brt-devkit-auth
aws sso login --profile jupiter_prod_engineer-425642425116
"""
import inspect
print(inspect.getmodule(Dataset))
print(LEFT_CAMERAS)
REV1_LEFT_CAMERAS = ['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left']
print(REV1_LEFT_CAMERAS)
print(ALL_CAMERA_PAIRS_LIST)

<module 'brtdevkit.data.dataset' from '/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py'>
['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left', 'T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02']
['front-center-left', 'front-left-left', 'front-right-left', 'side-left-left', 'side-right-left', 'rear-left']
[{'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}, {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03

In [3]:
# test_dataset = Dataset.retrieve(id='64ed2657926aefcd654e8269')
test_dataset = Dataset.retrieve(name='rev1_stops_2024_spring_for_labeling')
print(test_dataset.id)
test_df = test_dataset.to_dataframe()
print(test_df.shape)
if len(test_df) < 100000:
    test_df = test_df.sort_values('collected_on')
    print(test_df.iloc[0].collected_on, test_df.iloc[-1].collected_on)

673c019749f92b5b00a18709
(6102, 111)
2024-03-13T13:05:09.199000 2024-06-11T15:58:08.040000


In [6]:
# test_dfs = []
# test_dfs.append(test_df[test_df.camera_location.isin(['T09', 'T10', 'T11', 'T12'])])
test_dfs.append(test_df)
test_df = pd.concat(test_dfs, ignore_index=True)
test_df.shape

(3361, 182)

In [4]:
print(test_df.special_notes.unique())
print(test_df.iloc[0].id, test_df.iloc[-1].id)

AttributeError: 'DataFrame' object has no attribute 'special_notes'

In [5]:
test_df[['id', 'camera_location']].groupby('camera_location').size()
# test_df[['id', 'operation_time']].groupby('operation_time').size()

camera_location
T01    954
T02    952
T05    954
T06    953
T09    951
T10    951
T13    950
T14    950
dtype: int64

In [129]:
def get_chunk(df, t1, t2):
    df['collected_on_str'] = df['collected_on'].apply(lambda c: str(c)[:len(t1)])
    if not 'T' in df.iloc[0].collected_on_str:
        t1 = t1.replace('T', ' ')
        t2 = t2.replace('T', ' ')
    df2 = df[(df.collected_on_str >= t1) & (df.collected_on_str <= t2)]
    print(df.shape, df2.shape)
    print(df2.camera_location.unique())
    df3 = df2.groupby('frame_id').filter(lambda x: x['frame_id'].count() % 4 == 0)
    print(df.shape, df2.shape, df3.shape)
    return df3

t1, t2 = '2024-10-22T22:10:00', '2024-10-22T22:10:30'
df_sampled = get_chunk(df, t1, t2)
print(df_sampled.shape)

(74013, 14) (1488, 14)
['T02' 'T10' 'T13' 'T11' 'T08' 'T04' 'T03' 'T06' 'T05' 'T07' 'T16' 'T15'
 'T14' 'T12' 'T09' 'T01']
(74013, 14) (1488, 14) (1488, 14)
(1488, 14)


In [130]:
df_sampled.groupby('frame_id').size().to_frame('size').reset_index()['size'].to_list()
# df_sampled.groupby('frame_id').size()

[16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16]

In [127]:
df_sampled.iloc[0].robot_name

'1RW8410DTPK808012'

## Run Databricks Query

In [145]:
query = f"""
SELECT
  IMJ.id,
  IMJ.operation_time,
  IMJ.camera_location,
  IMJ.frame_id,
  IMJ.robot_name,
  IMJ.collected_on,
  IMJ.special_notes,
  IMJ.hard_drive_name,
  IMJ.spark_request__json,
  IMJ.teleop_request__json,
  IMJ.gps_can_data__json,
  IMJ.geohash,
  IMJ.autonomy_state__json
FROM
  mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
WHERE
  IMJ.collected_on > date('2024-05-31')
  AND IMJ.collected_on <= date('2024-06-30')
"""
  # IMJ.robot_name = '1RW8410DTPK808012'
  # AND 

df = query_db(query)
if len(df) > 0:
    print(df.shape, len(df)//2)
    if len(df) < 100000:
        df = df.sort_values('collected_on')
        df['collected_on_str'] = df['collected_on'].apply(lambda c: str(c)[:-13])
        print(df.iloc[0].collected_on_str, df.iloc[-1].collected_on_str)
        print(df.iloc[0].id, df.iloc[-1].id)

(10477657, 13) 5238828


In [43]:
special_notes = df2.special_notes.unique()
print(len(special_notes))
list(special_notes)

2


[None, 'human&&yoga-ball-42cm-30m']

In [20]:
# exclude train geohashes
def check_geohash_overlap(df, split=''):
    geohash_csv = '/data/jupiter/li.yu/data/halo_hard_cases/geohash_table_v11_4.csv'
    geohash_df = pd.read_csv(geohash_csv)
    print(geohash_df.shape)
    buckets = ['train', 'test', 'buffer']
    # get geohash6 of queried df
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    print(f'geohash table sample geohash6, {geohash_df.sample(5).geohash6.to_list()}')
    print(f'queried df sample geohash6, {df.sample(5).geohash6.to_list()}')
    # check overlap
    s = ''
    for b in buckets:
        bdf = df[df.geohash6.isin(geohash_df[geohash_df.bucket == b].geohash6)]
        s += f'{b} overlap: {len(bdf)}, '
        if b == split:
            df = bdf
            print('reduce df to df in test geohashs')
    print(s)
    return df

_ = check_geohash_overlap(df2)

(3478, 2)
geohash table sample geohash6, ['9vdcqm', '9zqs27', '9uftez', 'dp08s1', 'dp03h0']
queried df sample geohash6, ['dp7md2', 'dp7md6', 'dp7m6r', 'dp7md2', 'dp7md3']
train overlap: 0, test overlap: 0, buffer overlap: 0, 


In [24]:
df = df2
df.shape

(464710, 13)

In [148]:
# df2 = df[(df.special_notes.isin(['Human capture with dusty rear lenses', 'Dusty back cams human cont capture'])) & (df.camera_location.isin(['T09', 'T10']))]
# df2 = df[(~df.special_notes.isna()) & (df.special_notes.str.startswith('8RX 30'))]
# df2 = df[(df.camera_location.isin(REV1_LEFT_CAMERAS))]
# df2 = df[(df.camera_location.isin(['T09', 'T10', 'T11', 'T12'])) & (df.special_notes != 'test')]
df2 = df[(~df.spark_request__json.isna()) | (~df.teleop_request__json.isna())]
# df.special_notes.fillna('', inplace=True)
# df2 = df[df.special_notes.str.startswith('dust data collection')]
# df2 = df[(df.hard_drive_name == 'JUPD-1105_2024-10-23') | (df.special_notes == 'dust-left-pod-right -side')]
# df2 = df[df.robot_name.isin(['frodo2411'])]
# df2 = df[(df.collected_on_str > '2024-10-14 14:00:00') & (df.collected_on_str < '2024-10-14 14:40:00')]

# special_notes = df.special_notes.unique()
# # special_notes = [s for s in special_notes if s is not None and 'manure' in s.lower()]
# sps = []
# for s in special_notes:
#     for k in ['dust', 'smudge', 'jf163', 'jf149']:
#         if k in s.lower():
#             sps.append(s)
# print(sps)
# df2 = df[df.special_notes.isin(sps)]
# df2 = df[df.special_notes == 'tall stalks with blowing residue']

df.shape, df2.shape

((10477657, 13), (5447, 13))

In [75]:
def print_time(df):
    df = df.sort_values('collected_on')
    for i in range(0, len(df), len(df)//5):
        print(df.iloc[i].collected_on, df.iloc[i].id, df.iloc[i].special_notes)
print_time(seq_dfs[2])

2024-01-04 20:44:43.948000+00:00 65971ce8bad6bf531655d4b1 None
2024-01-04 20:44:52.792000+00:00 65971c82c7024818008f6479 None
2024-01-04 20:45:10.637000+00:00 65971ce6d76f2c1463c72b2f None
2024-01-04 20:45:14.988000+00:00 65971c53d76f2c1463c72a10 None
2024-01-04 20:45:19.496000+00:00 65971d3782f40946d133a1a6 None
2024-01-04 20:45:23.898000+00:00 65971d04fa73b2349c60296d None


In [82]:
# df2.groupby('special_notes').size()
df2.groupby('camera_location').size()

camera_location
front-center-left    549
front-left-left      527
front-right-left     548
rear-left            464
side-left-left       478
side-right-left      466
dtype: int64

In [149]:
df2.shape, df3.shape

((5447, 13), (9, 14))

In [150]:
df2 = pd.concat([df2, df3], ignore_index=True)
df2.shape

(5456, 14)

In [151]:
df2.to_csv('/data/jupiter/li.yu/data/halo_fps_cases/aletheia_stops_202406.csv', index=False)

### Sample df by camera pod or geohash & min

In [19]:
# sample by camera pod mentioned in special notes
pod_2_camera = {'front': ['T01', 'T02', 'T03', 'T04'], 'right': ['T05', 'T06', 'T07', 'T08'], 'back': ['T09', 'T10', 'T11', 'T12'], 'left': ['T13', 'T14', 'T15', 'T16']}
camera_2_pod = {c: p for p, cs in pod_2_camera.items() for c in cs}
ticket = 'JF163'
dfs = []
for s in sps:
    # if ticket in s:
    if not 'JF149' in s and not 'JF163' in s:
        # check for camera pairs mentioned in special note
        cs = []
        for c, p in camera_2_pod.items():
            if c in s:
                cs += pod_2_camera[p]
        # check for pod name mentioned
        for p, c in pod_2_camera.items():
            if p in s:
                cs += pod_2_camera[p]
        cs = list(set(cs))
        _df = df2[(df2.special_notes == s) & (df2.camera_location.isin(cs))]
        print(s, _df.shape)
        dfs.append(_df)
df_sampled = pd.concat(dfs, ignore_index=True)
len(dfs), df_sampled.shape

NameError: name 'sps' is not defined

In [18]:
# sample by vehicle speed, geohash, and collected min
def sample_by_speed_geohash6_collected_min(df, limit_by_min_geohash6, limit_by_geohash6=500):
    def get_vehicle_speed(d):
        if d is None:
            speed = 0
        else:
            speed = json.loads(d).get('speed', 0)
        return speed

    # only take left cameras
    # df2 = df[(df.camera_location.isin(['T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14']))]
    df2 = df[(df.camera_location.isin(LEFT_CAMERAS))]
    print(df.shape, df2.shape)

    if 'gps_can_data__json' in df2:
        print('# of images without gps_can_data__json', df2[df2.gps_can_data__json.isna()].shape)
        df2['speed'] = df2['gps_can_data__json'].apply(lambda d: get_vehicle_speed(d))
        df3 = df2[df2.speed > 0]
    else:
        df3 = df2
    print(df2.shape, df3.shape)
    
    df3['geohash6'] = df3['geohash'].apply(lambda g: g[:6])
    if limit_by_min_geohash6 > 0:  # limit # of images by geohash6 and time (total is 1440 per geohash6 & 1min)
        df3['collected_on_min'] = df3['collected_on'].apply(lambda c: str(c)[:16])
        df4 = df3.groupby(['geohash6', 'collected_on_min']).head(limit_by_min_geohash6).reset_index(drop=True)
    else:  # limit # of images only by geohash6
        assert limit_by_geohash6 > 0
        df4 = df3.groupby(['geohash6']).head(limit_by_geohash6).reset_index(drop=True)
    print(df4.shape)
    return df4

# load master df if needed
mdf = pd.read_csv('/data3/jupiter/datasets/model_positive/halo_human_w_corn_stubble_0812_oct/master_annotations.csv')
df = mdf[mdf.id.isin(df.id)].drop_duplicates(subset='id')

df_sampled = sample_by_speed_geohash6_collected_min(df, limit_by_min_geohash6=-1, limit_by_geohash6=500)

(2004, 156) (2004, 156)
(2004, 156) (2004, 156)
(1478, 157)


In [4]:
def print_geohash6_distribution(df):
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    print(df.groupby('geohash6').size())
print_geohash6_distribution(test_df)

geohash6
7zzzzz    624
9q9d4y      1
9q9d4z     54
9v6tps      4
9z5yqx     41
         ... 
dpksnu    119
dpkswg      4
dpkswu    138
dpksxh     38
dpku8j    137
Length: 150, dtype: int64


### Match query results with spark engagements

In [147]:
# select based on engagements
import ast
from collections import defaultdict

system = 'rev1' # rev1 or rev2
# load engagement csv
engagement_csv = '/data/jupiter/li.yu/data/halo_fps_cases/rev1_2024_sring_spark_engagements.csv'
engagement_df = pd.read_csv(engagement_csv, converters={"metadata": ast.literal_eval})
print('spark engagements', engagement_df.shape)
# get timestamps of each engagement
camera_time_d = defaultdict(list)
for i,row in engagement_df.iterrows():
    timestamps = row.metadata['media_timestamps']
    if system == 'rev1':
        camera_pair = tuple([row.metadata['camera_name'].replace('_', '-') + '-left',])  # works for rev1
        timestamps = [t[:-4] for t in timestamps]  # trim to 2024-06-04 22:44:31
    else:
        camera_pair = tuple(row.metadata['camera_name'][-7:].split('_'))  # works for rev2
        timestamps = [t[:-7] for t in timestamps]  # trim to 2024-06-04 22:44:31
    camera_time_d[camera_pair] += timestamps
for camera_pair in camera_time_d:
    ls = list(set(camera_time_d[camera_pair]))
    ls.sort()
    camera_time_d[camera_pair] = ls
# perform selection
def perform_selection(df):
    if not 'collect_on_str' in df:
        df['collected_on_str'] = df['collected_on'].apply(lambda c: str(c)[:-13])
    dfs = []
    for camera_pair, timestamps in camera_time_d.items():
        sub_df = df[(df.camera_location.isin(camera_pair)) & (df.collected_on_str.isin(timestamps))]
        dfs.append(sub_df)
    df2 = pd.concat(dfs, ignore_index=True)
    print('aletheia queries', df.shape)
    print('matched data in aletheia', df2.shape)
    return df2
df3 = perform_selection(df2)

spark engagements (4172, 3)
aletheia queries (4116, 14)
matched data in aletheia (9, 14)


/tmp/ipykernel_2650174/992961465.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['collected_on_str'] = df['collected_on'].apply(lambda c: str(c)[:-13])


In [109]:
seq_dfs = get_sequences(df2, interval=60)
print(len(seq_dfs))
for i, seq_df in enumerate(seq_dfs):
    print(i, seq_df.iloc[0].collected_on_str, seq_df.iloc[-1].collected_on_str)

10
0 2024-01-04 20:26:08 2024-01-04 20:26:23
1 2024-01-04 20:32:45 2024-01-04 20:33:00
2 2024-01-04 20:44:43 2024-01-04 20:45:23
3 2024-01-04 20:53:04 2024-01-04 20:53:19
4 2024-01-05 19:36:42 2024-01-05 19:37:51
5 2024-01-11 21:45:40 2024-01-11 21:45:50
6 2024-01-11 21:56:57 2024-01-11 21:58:18
7 2024-01-17 19:09:58 2024-01-17 19:10:13
8 2024-01-17 19:11:45 2024-01-17 19:13:43
9 2024-01-17 19:18:53 2024-01-17 19:19:08


In [89]:
camera_time_d

defaultdict(list,
            {('rear-left',): ['2024-01-04 20:26:12',
              '2024-01-04 20:26:13',
              '2024-01-04 20:44:47',
              '2024-01-04 20:44:48',
              '2024-01-04 20:44:49',
              '2024-01-08 17:38:57',
              '2024-01-08 17:38:58',
              '2024-01-08 17:38:59',
              '2024-01-08 17:39:13',
              '2024-01-08 17:39:14',
              '2024-01-08 17:42:38',
              '2024-01-08 17:42:39',
              '2024-01-08 17:46:51',
              '2024-01-08 17:46:52',
              '2024-01-08 17:47:03',
              '2024-01-08 17:47:04',
              '2024-01-08 17:47:05',
              '2024-01-08 19:00:11',
              '2024-01-08 19:00:12',
              '2024-01-08 19:00:13',
              '2024-01-08 19:01:38',
              '2024-01-08 19:01:39',
              '2024-01-08 19:01:40',
              '2024-01-08 19:36:58',
              '2024-01-08 19:36:59',
              '2024-02-22 18:43:38',
    

### Search around birds / airborne debris images

In [55]:
df['serial_number'] = df.calibration_data__json.apply(lambda x: json.loads(x).get('serial_number', np.nan))
df['sn7'] = df['serial_number'].apply(lambda s: s[:7])
df['geohash6'] = df['geohash'].apply(lambda s: s[:6])
df['sn7'].unique(), df['geohash6'].unique()

(array(['PCE7T1B', 'PCE7T1A'], dtype=object),
 array(['9z1pg7', '9z1pgk', '9q9d4w', '9ufs6w', '9z1pgj', '9z1pgh',
        '9ufs6y', '9ufs6v', '9z1pg5', '9vg20v', '9ufs6x', '9ufs6z',
        '9ufs6t', '9zmt8e', '7zzzzz', '9zmt8d', '9q9d4z', '9ufsdg',
        '9ufsd7', '9ufsd8', '9q9d4x', '9q9d4y', '9ufsd2', '9ze3m3',
        '9ze3m7', '9ze3m9', '9ze3me', '9ze3m6', '9ze3md', '9ze3mk',
        '9ze3ms', '9zmt8s', '9z1pyy', '9z1pzm', '9z1pzn', '9z1ptw',
        '9z1pzq', '9z1pzj', '9ze3m2', '9yyzmd', '9yyzm8', '9yyzjr',
        '9yyzm9', '9ufsd3', '9ufsd1', '9ufsd6'], dtype=object))

In [57]:
sbdf = df[df.sn7 == 'PCE7T1B']
sbdf.shape

(9020603, 30)

In [56]:
df.to_csv('/data/jupiter/li.yu/data/halo_hard_cases/all_mcap_data_till_0418.csv', index=False)
# df.to_csv('/data/jupiter/li.yu/data/halo_rgb_stereo_train_test/all_human_pose_0326_v4.csv', index=False)

In [ ]:
# read in geohash table
geohash_table = '/data/jupiter/li.yu/data/halo_hard_cases/20240328_geohash_table_v8_1_rc.csv'
geohash_df = pd.read_csv(geohash_table)
geohash6_train_list = geohash_df[geohash_df.bucket == 'train'].geohash6.to_list()
geohash6_test_list = geohash_df[geohash_df.bucket == 'test'].geohash6.to_list()
print(geohash_df.shape, len(geohash6_train_list), len(geohash6_test_list))

In [5]:
import ast
root_dir = '/data2/jupiter/datasets/'
# dataset = 'halo_rgb_stereo_train_v6_2'
dataset = 'halo_rgb_stereo_train_v8_1'
csv = os.path.join(root_dir, dataset, 'master_annotations.csv')
converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
# converters = {}
df = pd.read_csv(csv, converters=converters)

# ab_df = df[df.has_airborne == True]
# print(ab_df.shape, ab_df.iloc[0].geohash)
# ab_df["geohash_6"] = ab_df["geohash"].apply(lambda x: x[:6])

df['Birds'] = df['label_counts'].apply(lambda d: d.get('Birds', 0))
df['has_birds'] = df['Birds'] > 0
bird_df = df[df.has_birds == True]
print(bird_df.shape, bird_df.iloc[0].geohash)
bird_df["geohash_6"] = bird_df["geohash"].apply(lambda x: x[:6])

from jupiterdata.utils.dataset import query_db

(1362, 361) 9xjg9cde9btm


/tmp/ipykernel_103657/3024971100.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bird_df["geohash_6"] = bird_df["geohash"].apply(lambda x: x[:6])


In [7]:
LEFT_CAMERAS = ('T01', 'T02', 'T05', 'T06', 'T09', 'T10', 'T13', 'T14', 'I01', 'I02')
geohash_6_limit = 500
geohash_6_expand = 1.5
surround_ids = []
for geohash_6 in bird_df.geohash_6.unique():
    collected_on_days = bird_df[(bird_df.geohash_6 == geohash_6)].collected_on_day.unique()
    for collected_on_day in collected_on_days:
        current = len(bird_df[(bird_df.geohash_6 == geohash_6) & (bird_df.collected_on_day == collected_on_day)].unique_id.unique())
        expand = int(geohash_6_limit / current * geohash_6_expand)
        if expand < 50:
            expand = 50
        elif expand > 500:
            expand = 500

        query = f"""
        SELECT
          IMJ.id AS id,
          IMJ.robot_name AS robot_name,
          IMJ.camera_location AS camera_location,
          IMJ.operating_field_name AS operating_field_name,
          IMJ.bag_name AS bag_name,
          IMJ.collected_on AS collected_on,
          IMJ.hard_drive_name AS hard_drive_name,
          IMJ.geohash AS geohash
        FROM
          mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
        WHERE
          collected_on LIKE '{collected_on_day[:10]}%'
          AND geohash LIKE '{geohash_6}%'
          AND camera_location IN {LEFT_CAMERAS}
        LIMIT {expand};
        """
        df = query_db(query)
        surround_ids += df.id.to_list()
        print(geohash_6, collected_on_day, current, len(df))
    #     break
    # break
    # print('queried geohash_6', geohash_6)

len(surround_ids)

9xjg9c 2023-08-04 13 57
9v5ks7 2023-08-02 153 50
9v5kwy 2023-08-05 2 375
9v5kxj 2023-08-07 20 50
9v5kse 2023-08-02 67 50
9xjg3z 2023-08-05 4 187
9v5kss 2023-08-02 32 50
dp03h5 2023-09-08 3 250
9v5ksk 2023-08-02 22 50
9xjgd0 2023-08-04 4 187
9xjgd0 2023-08-05 1 500
9xjgy5 2023-08-15 8 93
9v5kwu 2023-08-05 9 83
dp03ht 2023-09-09 1 500
9xjg9b 2023-08-04 5 150
9xjg9b 2023-08-05 2 375
9xjg92 2023-07-28 8 93
9xjepp 2023-08-09 2 375
dju21r 2023-08-18 2 375
9v5kxn 2023-08-07 7 107
9xjg3f 2023-08-07 5 150
9xjgy7 2023-08-15 2 375
9xjg8b 2023-08-17 4 187
dju21p 2023-08-18 4 187
9xjg98 2023-07-28 2 375
dp03k9 2023-09-13 3 250
9xjg64 2023-08-07 1 500
9xjg3d 2023-08-07 2 375
9xjgvu 2023-08-15 3 31
9v5kws 2023-08-07 2 375
9v5kwv 2023-08-05 19 50
9xjg9m 2023-08-16 5 150
dp03hx 2023-09-12 4 187
9xjg3r 2023-08-04 1 500
9xjg3r 2023-08-16 1 500
9v5ksd 2023-08-02 5 150
9xjer0 2023-08-09 1 500
dp03k0 2023-09-11 3 250
9v5ks6 2023-08-01 2 375
9v5ks6 2023-08-02 18 50
9xjg6p 2023-08-05 1 500
9xjg3g 2023-08-07 1

60573

In [8]:
remain_ids = list(set(surround_ids) - set(df.id.to_list()))
print(len(remain_ids))
remain_ids[0]

60073


'654658b46c228e535a897327'

### Search around a time

In [3]:
from datetime import datetime, timedelta
from jupiterdata.utils.dataset import query_db

In [36]:
terrain_csv = '/data/jupiter/li.yu/data/dune_simulation/representative_terrains.csv'
terrain_df = pd.read_csv(terrain_csv)
terrain_df['datetime'] = terrain_df.collected_on.apply(datetime.fromisoformat)
delta = timedelta(seconds=2)
print(terrain_df.shape)
terrain_df.head(2)

(15, 5)


,id,collected_on,bag_name,terrain,datetime
0,65eb4802738e53d66c8f346a,2024-02-08T19:22:19.435000,02_08_2024-19_22_27,open_field,2024-02-08 19:22:19.435
1,65fb4c39ae44446ff5d3fc19,2024-02-23T21:18:43.051000,02_23_2024-21_18_45,in_farm,2024-02-23 21:18:43.051


In [40]:
str(terrain_df.iloc[-1].datetime), str(terrain_df.iloc[-1].datetime + delta)[:19]

('2023-10-02 17:39:21.561000', '2023-10-02 17:39:23')

In [37]:
dfs = []
for i,row in terrain_df.iterrows():

    query = f"""
    SELECT
        IMJ.id AS id,
        IMJ.robot_name AS robot_name,
        IMJ.camera_location AS camera_location,
        IMJ.operating_field_name AS operating_field_name,
        IMJ.bag_name AS bag_name,
        IMJ.collected_on AS collected_on,
        IMJ.hard_drive_name AS hard_drive_name,
        IMJ.geohash AS geohash
    FROM
        mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
    WHERE
        bag_name = '{row.bag_name}'
        AND collected_on > '{str(row.datetime - delta)[:19]}'
        AND collected_on <= '{str(row.datetime + delta)[:19]}'
    """

    df = query_db(query)
    print(row.id, row.terrain, len(df))
    dfs.append(df)
    # break

65eb4802738e53d66c8f346a open_field 88
65fb4c39ae44446ff5d3fc19 in_farm 192
65eb1913abb27b5979bb2c49 trees 96
65fb5e03fbbd37ea5e01a225 pole_by_road 191
65fb3f89c7d378dbc65eb080 birds 192
6516a9ee59d4a80830b8f3f2 head_lands 32
64e5d2fd7d3e736bc96a7e64 tall_weeds 80
653b8ab65516ee030864597f irrigation 32
650420e64f21186e0f2bda70 road_side 44
655b1dfb1ec667f4ad177ea1 tall_weeds2 32
65fb3235e9fc675f8fee1f38 dust 192
654d7f53443a59d899337355 flooded_area 60
651f4ca5397e2827e807f262 head_lands2 18
6531b6fa44af03aa185c2c36 head_lands3 48
651f586e7747a76b7ba779a8 open_field2 32


In [38]:
df = df.sort_values('collected_on')
df[['id', 'collected_on']]

,id,collected_on
11,651f53a5d2dd50c6782fc604,2023-10-02 17:39:19.451000+00:00
20,651f53a8d7bd74435539150e,2023-10-02 17:39:19.487000+00:00
4,651f53a6f5298a1f9837649e,2023-10-02 17:39:19.513000+00:00
30,651f53aa4393288ab0595044,2023-10-02 17:39:19.523000+00:00
7,651f53ad1fc961b2c883145a,2023-10-02 17:39:19.528000+00:00
31,651f53b188a7ef1b7c505df2,2023-10-02 17:39:19.538000+00:00
18,651f53af24484d263bf73813,2023-10-02 17:39:19.545000+00:00
12,651f53ab66fd3736ba6d439d,2023-10-02 17:39:19.554000+00:00
5,651f57e52945e575a12a2c3a,2023-10-02 17:39:20.470000+00:00
24,651f57ec234278694e6702c2,2023-10-02 17:39:20.503000+00:00


In [41]:
df = pd.concat(dfs, ignore_index=True)
print(df.shape)

(1329, 8)


### Search around robot and time

In [11]:
from datetime import datetime, timedelta
from jupiterdata.utils.dataset import query_db

In [12]:
test_df['datetime'] = test_df.collected_on.apply(datetime.fromisoformat)
delta = timedelta(seconds=10)
test_df.shape

(146, 183)

In [14]:
dfs = []
for i, row in test_df.iterrows():
    query = f"""
    SELECT
        IMJ.id AS id,
        IMJ.robot_name AS robot_name,
        IMJ.camera_location AS camera_location,
        IMJ.operating_field_name AS operating_field_name,
        IMJ.bag_name AS bag_name,
        IMJ.collected_on AS collected_on,
        IMJ.hard_drive_name AS hard_drive_name,
        IMJ.geohash AS geohash
    FROM
        mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
    WHERE
        robot_name = '{row.robot_name}'
        AND collected_on >= '{str(row.datetime - delta)[:19]}'
        AND collected_on <= '{str(row.datetime + delta)[:19]}'
        AND camera_location = '{row.camera_location}'
    """

    df = query_db(query)
    print(i, row.robot_name, row.collected_on, row.camera_location, len(df))
    dfs.append(df)
    # break

0 halohitchhiker_122 2023-08-04T03:23:37.406000 T14 11
1 halohitchhiker_112 2023-08-05T01:24:25.348000 T14 11
2 halohitchhiker_102 2023-09-10T01:11:32.949000 T14 11
3 halohitchhiker_102 2023-09-10T01:36:33.994000 T14 11
5 halohitchhiker_102 2023-09-15T00:09:46.611000 T09 11
4 halohitchhiker_102 2023-09-15T00:09:48.636000 T09 11
7 halohitchhiker_102 2023-09-15T00:13:50.578000 T10 11
6 halohitchhiker_102 2023-09-15T00:13:52.591000 T10 11
8 halohitchhiker_101 2023-10-03T00:48:55.802000 T06 11
9 halohitchhiker_101 2023-10-04T23:54:58.063000 T05 12
11 halohitchhiker_101 2023-10-21T23:14:58.914000 T05 11
12 halohitchhiker_101 2023-10-21T23:14:59.908000 T05 11
10 halohitchhiker_101 2023-10-21T23:15:03.911000 T05 11
13 halohitchhiker_101 2023-10-21T23:38:17.923000 T05 11
14 halohitchhiker_152 2023-10-24T03:39:14.385000 T10 12
15 halohitchhiker_112 2023-10-31T12:35:58.892000 T09 11
16 halohitchhiker_112 2023-10-31T12:36:01.901000 T09 11
17 halohitchhiker_112 2023-10-31T12:36:14.880000 T09 11
19

In [17]:
fdf = pd.concat(dfs, ignore_index=True)
fdf.drop_duplicates(subset=['id'], inplace=True)
fdf = fdf[(fdf.camera_location.isin(LEFT_CAMERAS)) & (~fdf.id.isin(test_df.id))]
fdf.shape

(3219, 8)

### Search around geohash and time

In [7]:
import ast
from datetime import datetime, timedelta
from jupiterdata.utils.dataset import query_db

def get_fn_geohash6_bag_name(root_dir, pred_dir, model, dataset):
    # load dataset df
    csv = os.path.join(root_dir, dataset, 'master_annotations.csv')
    converters = {"label_map": ast.literal_eval, "label_counts": ast.literal_eval}
    df = pd.read_csv(csv, converters=converters)
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    print(df.shape)

    # load pred fns
    sum_pred_dir = os.path.join(pred_dir, model, dataset, 'false_negative_sum5')
    fn_ids = os.listdir(sum_pred_dir)
    unique_ids = [f[:-4] for f in fn_ids]

    # get fn df
    fn_df = df[df.unique_id.isin(unique_ids)]
    return fn_df

In [25]:
# read in geohash table
geohash_table = '/data/jupiter/li.yu/data/halo_hard_cases/20240328_geohash_table_v8_1_rc.csv'
geohash_df = pd.read_csv(geohash_table)
geohash6_train_list = geohash_df[geohash_df.bucket == 'train'].geohash6.to_list()
geohash6_test_list = geohash_df[geohash_df.bucket == 'test'].geohash6.to_list()
print(geohash_df.shape, len(geohash6_train_list), len(geohash6_test_list))

(2101, 2) 1672 371


In [8]:
mcsv = '/data2/jupiter/datasets/halo_rgb_stereo_train_v8_1/master_annotations.csv'
mdf = pd.read_csv(mcsv)
print(mdf.shape)

root_dir = '/data/jupiter/datasets/safety_datasets/'
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
model = 'v81_nextvitsmall_dust_only_kore_dean_0406'
datasets = [
    "humans/on_path_forward/on_path_forward_humans_night_2024_rev2_v10", 
    "humans/on_path_aft/on_path_aft_humans_night_2024_rev2_v10",
]
fn_dfs = [get_fn_geohash6_bag_name(root_dir, pred_dir, model, dataset) for dataset in datasets]
fn_df = pd.concat(fn_dfs, ignore_index=True)
print(fn_df.shape)
print(len(fn_df.bag_name.unique()), len(fn_df.geohash6.unique()))

(326871, 359)


/tmp/ipykernel_121511/3595870924.py:9: DtypeWarning: Columns (23,37,132,150,168,184,188,196,218,219) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv, converters=converters)


(4251, 332)
(2260, 332)
(651, 333)
187 23


/tmp/ipykernel_121511/3595870924.py:9: DtypeWarning: Columns (164,165,172,174,184,213,218,219,222,224,239,273,274) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv, converters=converters)


In [9]:
g = fn_df.groupby(['geohash6', 'bag_name']).size()
keys = g.index
d = [[k, g[k]] for k in keys]
d = sorted(d, key=lambda k: k[1], reverse=True)
d[:5]

[[('9v6tpg', '02_14_2024-00_50_50'), 27],
 [('9zpund', '10_21_2023-00_44_44'), 17],
 [('9v6tps', '02_21_2024-01_24_00'), 16],
 [('9zsx7q', '11_06_2023-23_46_58'), 15],
 [('9v6tpg', '02_17_2024-01_35_37'), 14]]

In [14]:
# get collection time of fn images in a bag
def get_start_end_time_of_bag(fn_df, bag_name):
    # bag_name = '02_21_2024-01_24_00'
    bag_df = fn_df[fn_df.bag_name == bag_name]
    bag_df = bag_df.sort_values('collected_on')
    bag_df['datetime'] = bag_df.collected_on.apply(datetime.fromisoformat)
    delta = timedelta(seconds=60*30)
    # print(bag_df.shape)
    # print(bag_df.iloc[0].datetime, bag_df.iloc[-1].datetime)
    # print(bag_df.iloc[0].datetime - delta, bag_df.iloc[-1].datetime + delta)
    return bag_df.iloc[0].datetime - delta, bag_df.iloc[-1].datetime + delta

In [19]:
dfs = {}
for (geohash6, bag_name), N in d:
    t1, t2 = get_start_end_time_of_bag(fn_df, bag_name)

    query = f"""
    SELECT
        IMJ.id AS id,
        IMJ.robot_name AS robot_name,
        IMJ.camera_location AS camera_location,
        IMJ.operating_field_name AS operating_field_name,
        IMJ.bag_name AS bag_name,
        IMJ.collected_on AS collected_on,
        IMJ.hard_drive_name AS hard_drive_name,
        IMJ.geohash AS geohash
    FROM
        mesa_prod.mesa_lake_prod.image_jupiter AS IMJ
    WHERE
        bag_name = '{bag_name}'
        AND collected_on > '{str(t1)[:19]}'
        AND collected_on <= '{str(t2)[:19]}'
    """

    df = query_db(query)
    print(geohash6, bag_name, t1, t2, len(df))
    dfs[(geohash6, bag_name)] = df
    # break

9v6tpg 02_14_2024-00_50_50 2024-02-14 00:21:43.138000 2024-02-14 01:22:19.165000 2078
9zpund 10_21_2023-00_44_44 2023-10-21 00:15:52.182000 2023-10-21 01:16:38.163000 921
9v6tps 02_21_2024-01_24_00 2024-02-21 00:54:22.037000 2024-02-21 01:54:43.033000 3377
9zsx7q 11_06_2023-23_46_58 2023-11-06 23:17:33.235000 2023-11-07 00:18:47.270000 1048
9v6tpg 02_17_2024-01_35_37 2024-02-17 01:06:00.170000 2024-02-17 02:07:00.095000 1616
9v6tpg 02_21_2024-01_02_22 2024-02-21 00:33:58.630000 2024-02-21 01:34:45.661000 2062
9v6tpg 02_24_2024-00_54_03 2024-02-24 00:24:42.477000 2024-02-24 01:25:06.534000 1459
9zmjkf 11_18_2023-02_58_58 2023-11-18 02:30:22.710000 2023-11-18 03:30:47.680000 960
9zmjkf 11_17_2023-23_45_45 2023-11-17 23:16:29.858000 2023-11-18 00:17:37.874000 921
9v6tpg 02_25_2024-01_27_28 2024-02-25 00:58:04.455000 2024-02-25 01:58:22.454000 959
9v6tpg 02_14_2024-00_52_52 2024-02-14 00:23:13.958000 2024-02-14 01:23:32.119000 1182
9v6tpg 02_14_2024-03_00_10 2024-02-14 02:30:58.970000 2024

In [40]:
total = 0
use_df = None
for (geohash6, bag_name), df in dfs.items():
    df['geohash6'] = df['geohash'].apply(lambda g: g[:6])
    # _df = df[(df.geohash6 != '7zzzzz') & (~df.geohash6.isin(geohash6_test_list))]
    _df = df[(~df.geohash6.isin(geohash6_test_list))]
    if len(_df) > 0:
        use_df = _df if len(_df) > 200 else use_df
        total += len(_df)
        print(geohash6, bag_name, _df.shape)
        print(_df.sample(min(5, len(_df))).id.to_list())
        # if len(_df[_df.id == '654bdc10851dd6051c9a2d4b']) > 0:
        #     print(_df[_df.id == '654bdc10851dd6051c9a2d4b'].geohash.to_list())
print(total)

9v6tpg 02_14_2024-00_50_50 (554, 9)
['65ea454263edec78ede83128', '65ea42b9936e32acd000c032', '65ea4a7c000669bb859209b4', '65ea482b71e1684babf8f77f', '65ea462bbf9244df1b4d852e']
9v6tps 02_21_2024-01_24_00 (904, 9)
['65e9cdab3de3d70fe449654b', '65ea0b36000669bb8591757d', '65e9d226e879f487a282a835', '65e9d004cafc4e811b02af3d', '65e9d12bf2f15c5f2ceaa879']
9zsx7q 11_06_2023-23_46_58 (88, 9)
['654bdc741094059fbafd8f00', '654bdc6334b229e20d4375a9', '654bdc2200210142b9092ba1', '654bdbd9fa83693bde13c334', '654bdc32f99b22029bd77dbb']
9v6tpg 02_17_2024-01_35_37 (445, 9)
['65e9c01ebc620c2c14fe4026', '65e9c0945e301af12dab0518', '65e9c12a2bbbdbae66cedae3', '65e9bdb5f04f29f44a32c6ab', '65e9c0784875033da61a0078']
9v6tpg 02_21_2024-01_02_22 (539, 9)
['65ea06aedb75a515c2d7ad48', '65e9ffc6459275487d44b70b', '65ea00ca6592f96a05175cee', '65ea01c0d80ab49742253a49', '65ea06f706198505e2a64568']
9v6tpg 02_24_2024-00_54_03 (396, 9)
['65e7c3694875033da61782fb', '65e7c80ddc04dc9d140eb419', '65e7c5a94875033da61787

In [41]:
use_df.geohash6.unique()

array(['7zzzzz'], dtype=object)

## Create a dataset

### Some filtering

In [47]:
# model false positives
pred_dir = '/data/jupiter/li.yu/exps/driveable_terrain_model'
models = [
    'kore_5_h',
    'kore_10_lo_10_tr',
]
dataset = 'fps_datasets/halo_pivot_voltage_culvert_data_8012_oct'
fp_ids, fp_uids = [], []
for model in models:
    pred_df = pd.read_csv(os.path.join(os.path.join(pred_dir, model, dataset, 'output.csv')))
    fp_df = pred_df[pred_df.result_state == 'false_positive']
    print(model, pred_df.shape, fp_df.shape)
    fp_ids += fp_df.id.to_list()
    fp_uids += fp_df.unique_id.to_list()
fp_ids = list(set(fp_ids))
fp_uids = list(set(fp_uids))
print(len(fp_ids), len(fp_uids))

kore_5_h (56065, 23) (7425, 23)
kore_10_lo_10_tr (56065, 23) (7987, 23)
6707 8009


In [38]:
df3 = df2[df2.id.isin(fp_ids)]
print(df2.shape, df3.shape)
df3.special_notes.unique()

(74864, 12) (6707, 12)


array(['jason zaman high voltage lines', 'Pivot House',
       'irrigation hut + water tank', 'pivot headland pass',
       'irrigation pivot headland', 'pivot headlands',
       'Jason Zaman High voltage Lines', 'Road Culvert left of tractor'],
      dtype=object)

In [49]:
mdf = pd.read_csv('/data2/jupiter/datasets/fps_datasets/halo_pivot_voltage_culvert_data_8012_oct/master_annotations.csv')
fp_mdf = mdf[mdf.unique_id.isin(fp_uids)]
print(mdf.shape, fp_mdf.shape)
fp_mdf.to_csv('/data2/jupiter/datasets/fps_datasets/halo_pivot_voltage_culvert_data_8012_oct/master_annotations_fps.csv', index=False)

/tmp/ipykernel_1200557/2618062599.py:1: DtypeWarning: Columns (103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  mdf = pd.read_csv('/data2/jupiter/datasets/fps_datasets/halo_pivot_voltage_culvert_data_8012_oct/master_annotations.csv')


(56065, 155) (8009, 155)


In [13]:
# dedup by image similarity
# dedup_df = pd.read_csv('/data2/jupiter/datasets/fps_datasets/halo_pivot_voltage_culvert_data_8012_oct/image_similarity_reduced_1.csv')
dedup_df = pd.read_csv('/data3/jupiter/datasets/model_positive/halo_manure_data_candidate/image_similarity_clustered.csv')
print(dedup_df.shape)

(32190, 116)


In [152]:
# csv = '/data3/jupiter/datasets/model_positive/halo_human_w_corn_stubble_0812_oct/selected_for_label.csv'
# csv = '/data/jupiter/datasets/dust_datasets/halo_dust_smudge_JUPD-1105_2024-10-23/selected_for_label.csv'
csv = '/data/jupiter/li.yu/data/halo_fps_cases/rev1_2024_sring_stops_not_in_train.csv'
df = pd.read_csv(csv)
print(df.shape, 'id' in df, 'unique_id' in df)

(6104, 14) True False


In [156]:
df = df[~df.id.isin(test_df.id)]
df.shape

(6102, 14)

In [10]:
triage_ids = ['66a84455f9807557db40d8fc',
 '66ce211cf652389e420cd5c1']
print(len(triage_ids))
df = df[~df.id.isin(triage_ids)]
df.shape

2


(2293, 221)

In [8]:
# dfs = []
dfs.append(df)
df2 = pd.concat(dfs, ignore_index=True)
df2.shape

(2003, 1)

### Dataset creation

In [157]:
# unlabeled dataset
Dataset.create(
    name="rev1_stops_2024_spring_for_labeling",
    description="6102 images not in training set and unlabeled",
    kind='image',  # annotation or image
    image_ids=df.id.to_list())

{}

In [154]:
# labeled, stereo dataset
imageids_to_dataset(
    image_ids=df.id.to_list(),
    dataset_name="rev1_stops_2024_spring_labeled",
    dataset_description="6104 images not in training set",
    dataset_kind='pixelwise_annotation',  # image or categorical_annotation or pixelwise_annotation
    mode='stereo',  # stereo or mono
    production_dataset=False,
)

Warning 6102 images do not have a corresponding annotation.
Preparing stereo dataframe for {'front-center-left': 'front-center-right', 'front-left-left': 'front-left-right', 'front-right-left': 'front-right-right', 'side-left-left': 'side-left-right', 'side-right-left': 'side-right-right', 'rear-left': 'rear-right', 'front-center-right': 'front-center-left', 'front-left-right': 'front-left-left', 'front-right-right': 'front-right-left', 'side-left-right': 'side-left-left', 'side-right-right': 'side-right-left', 'rear-right': 'rear-left'}...
Size of left dataframe: 2
Size of stereo dataframe: 2
Sending 2 annotated_ids for creating dataset
Time taken to prepare data for dataset creation job: 0.27 mins


In [131]:
# unlabeled, stereo dataset
imageids_to_dataset(
    image_ids=df_sampled.id.to_list(),
    dataset_name="sequence_day_no_dust_driving",
    dataset_description="1488 images sampled from halo_manure_data_candidate, '2024-10-22T22:10:00', '2024-10-22T22:10:30'",
    dataset_kind='image',  # image or categorical_annotation or pixelwise_annotation
    mode='stereo',  # stereo or mono
    production_dataset=False,
)

Preparing stereo dataframe for {'T01': 'T03', 'T02': 'T04', 'T05': 'T07', 'T06': 'T08', 'T09': 'T11', 'T10': 'T12', 'T13': 'T15', 'T14': 'T16', 'I01': 'I03', 'I02': 'I04'}...
Size of left dataframe: 744
Size of stereo dataframe: 744
Preparing stereo dataframe for {'T02': 'T03', 'T06': 'T07', 'T10': 'T11', 'T14': 'T15', 'I02': 'I03'}...
Size of left dataframe: 372
Size of stereo dataframe: 372
Sending 1488 image ids for creating dataset
Time taken to prepare data for dataset creation job: 0.45 mins


## Download a dataset

In [45]:
# to download large image dataset and save in jpg format, change the following files
# note original files are ending with .bk, and changed files are ending with .save_jpg
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/data/core/dataset.py
# /home/li.yu/anaconda3/envs/pytorchlightning/lib/python3.8/site-packages/brtdevkit/util/aws/s3.py
# /home/li.yu/anaconda3/envs/brtdevkit/lib/python3.9/site-packages/brtdevkit/data/core/dataset.py  # save only one png file

In [6]:
dataset_name = 'Jupiter_1RW8410DLPB228592_20241016_human_dust_left'
dataset_dir = os.path.join('/data/jupiter/datasets/dust_datasets', dataset_name)
# dataset_dir = os.path.join('/data2/jupiter/datasets', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/li.yu/data/dust_data_colletion_for_july_1st', dataset_name)
# dataset_dir = os.path.join('/data/jupiter/datasets/image_quality_datasets', dataset_name)
# dataset_dir = os.path.join('/data3/jupiter/datasets/model_positive', dataset_name)
os.makedirs(dataset_dir, exist_ok=True)

test_dataset = Dataset.retrieve(name=dataset_name)
test_df = test_dataset.to_dataframe()
test_df.drop(columns=["artifact_raw_0_save_path"], inplace=True, errors="ignore")  # don't download tiff files
test_dataset.download(dataset_dir, df=test_df)

2024-11-14 08:06:54,352 - brtdevkit - INFO - start downloading dataset: 6734f0bce652e258df6210a3 into /data/jupiter/datasets/dust_datasets/Jupiter_1RW8410DLPB228592_20241016_human_dust_left
/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py:619: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keys_with_save_path = keys_with_save_path.append(
/home/li.yu/anaconda3/envs/query/lib/python3.10/site-packages/brtdevkit/data/dataset.py:619: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keys_with_save_path = keys_with_save_path.append(
100%|██████████| 15229/15229 [03:09<00:00, 80.54it/s] 
2024-11-14 08:10:08,482 - brtdevkit - INFO - finished downloading dataset: 6734f0bce652e258df6210a3
